# <div align="center"><b> Trabajo Final - Visión por computadora 2  - CEIA </b></div>

<div align="right">📝 <em><small><font color='Gray'>Nota:</font></small></em></div>

<div align="right"> <em><small><font color='Gray'> La funcionalidad de visualización de jupyter notebooks en <a href="https://github.com/" target="_blank">github</a> es solamente un preview.</font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Para mejor visualización se sugiere utilizar el visualizador recomndado por la comunidad: <a href="https://nbviewer.org/" target="_blank">nbviewer</a></font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Puedes a acceder al sigiente enlace para ver este notebook en dicha página: <a href="https://nbviewer.org/github/brunomaso1/uba-ceia/blob/ceia-nlp/ceia-nlp/Desafio%201.ipynb">Desafio 1</a></font></small></em> </div> 

<!-- TODO: Arreglar el link -->


* * *

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>

<!-- Colab -->
<!-- <div align="center"><img src="https://drive.google.com/uc?export=view&id=1tUnWeK372o_yu0LxkMwtC4IR2jorDipN" width="600" alt="Figura 1: Detección de palmeras mediante modelos de aprendizaje profundo."></div> -->

<div align="center"><img src="./resources/portada.jpeg" width="600" alt="Figura 1: Detección de palmeras mediante modelos de aprendizaje profundo."></div>

<div align="center"><small><em>Figura 1: Detección de palmeras mediante modelos de aprendizaje profundo.</em></small></div>

<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | Trabajo final - VPC2 - FIUBA                                                                                                           |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Descrpción**  | Palm tree detector                                                                                                                     |
| **Integrantes** | - Bruno Masoller (brunomaso1@gmail.com) </br> - Simón Rodriguez (simon.andre.r@gmail.com)                                              |

</div>

✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋

Inicialmente, cargamos los datos desde Google Drive (si no está disponible el link, se puede desacargar desde Kaggle también -hay información de esto más adelante-):

In [4]:
import os
import warnings

# Definimos la ruta de la carpeta y el archivo de requerimientos
CARPETA_RESOURCES = './resources'
archivo_requerimientos = os.path.join(CARPETA_RESOURCES, 'requirements.txt')

# Descargamos la carpeta con archivos auxiliares
%pip install gdown
!gdown https://drive.google.com/drive/folders/1DFfgoGQ8-zLwQU85Uyq0X72GX4IITInV?usp=sharing --folder

# Verificamos si la carpeta ha sido descargada correctamente
if os.path.exists(CARPETA_RESOURCES):
    print(f"La carpeta {CARPETA_RESOURCES} fue encontrada.")

    # Verificamos si el archivo de requerimientos existe
    if os.path.exists(archivo_requerimientos):
        print("Archivo de requerimientos encontrado, instalando dependencias...")
        %pip install -r {archivo_requerimientos}
    else:
        warnings.warn(f"El archivo {archivo_requerimientos} no fue encontrado. No se instalarán los requerimientos.")
else:
    warnings.warn(f"La carpeta {CARPETA_RESOURCES} no fue encontrada. Verifique la descarga.")

Note: you may need to restart the kernel to use updated packages.
Retrieving folder 1OfDuCFXsTIGBNyfX4w-3UqLfH7-ZXZPt datasets
Retrieving folder 1kBbFvw6DMuwvIO51tshEeFgRsq4ak3-6 compressed
Processing file 1zSMCq0xHcfSidJFCKg1AJtgWlPVhAwUH palmeras.zip
Processing file 1tUnWeK372o_yu0LxkMwtC4IR2jorDipN portada.jpeg
La carpeta ./resources fue encontrada.


Retrieving folder contents
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1zSMCq0xHcfSidJFCKg1AJtgWlPVhAwUH
From (redirected): https://drive.google.com/uc?id=1zSMCq0xHcfSidJFCKg1AJtgWlPVhAwUH&confirm=t&uuid=3ae8104b-4ce5-4afb-a6ba-ff2c9b3238bc
To: d:\Documentos\Git Repositories\uba-ceia\ceia-vpc2\resources\datasets\compressed\palmeras.zip

  0%|          | 0.00/818M [00:00<?, ?B/s]
  0%|          | 1.05M/818M [00:00<01:28, 9.25MB/s]
  1%|          | 4.19M/818M [00:00<00:37, 21.5MB/s]
  1%|          | 6.82M/818M [00:00<00:47, 17.1MB/s]
  1%|          | 8.91M/818M [00:00<00:48, 16.6MB/s]
  1%|▏         | 11.0M/818M [00:00<00:49, 16.2MB/s]
  2%|▏         | 14.7M/818M [00:00<00:38, 20.7MB/s]
  2%|▏         | 17.3M/818M [00:01<00:47, 16.9MB/s]
  3%|▎         | 21.0M/818M [00:01<00:38, 20.6MB/s]
  3%|▎         | 23.6M/818M [00:01<00:46, 17.2MB/s]
  3%|▎         | 26.7M/818

In [6]:
import zipfile
import torch

🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


In [3]:
# Parámetros
VERBOSE = True # Muestra época a época la evolución
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

CARPETA_RESOURCES = './resources'
CARPETA_COMPRESSED = './resources/datasets/compressed'
CARPETA_RAW = './resources/datasets/raw'

print(f'Dispositivo actual: {DEVICE}')

Dispositivo actual: cpu


## Introduction

En el marco de la especialización en Inteligencia Artificial de la Facultad de Ingeniería de la Universidad de Buenos Aires, se plantea, como trabajo final de la materia Visión por Computadora 2, desarrollar un modelo de visión por computadora que utilice aprendizaje profundo para detectar las ubicaciones de palmeras en diversas imágenes áereas.

Metodológicamente, se aplican principios de [CRISP-ML(Q)](https://ml-ops.org/content/crisp-ml) para el desarrollo del problema.

## Business and data understanding

La detección de palmeras es ampliamente estudiado en el área de visión por computadora, dado su potencial como insumo de otras soluciones, como por ejemplo, aquellas que intentan mejorar la salud de las palmeras, donde inicalmente es deseable una detección aera para luego realizar una inspección manual (con el objetivo de reducir costos).

Muchos estudios (como [este](https://www.researchgate.net/publication/382742087_Implementation_of_Slicing_Aided_Hyper_Inference_SAHI_in_YOLOv8_to_Counting_Oil_Palm_Trees_Using_High-Resolution_Aerial_Imagery_Data) por ejemplo) se realizan en paises en donde la mayoría de las palmeras son materia prima para la producción del "aceite de palmera". Sin embargo, en paises no productores, el cuidado de las palmeras también es importante, dado su impacto en el medio ambiente. 

## Data preparation

Los datos fueron obenidos del sito de Kaggle: https://www.kaggle.com/datasets/riotulab/aerial-images-of-palm-trees, sin embargo, para una mejor manejo, se los importan desde Google Drive (https://drive.google.com/drive/folders/1DFfgoGQ8-zLwQU85Uyq0X72GX4IITInV)

### Data visualization

Luego de descargado los datos, tenemos la siguiente estructura de directorios:

<!-- https://tree.nathanfriend.io/
- resources
  - datasets
    - compressed
      - palmeras_kaggle.zip
 -->

<pre>
resources/
└── datasets/
    └── compressed/
        └── palmeras_kaggle.zip
</pre>

Descomprimimos los datos:

In [7]:
# Crear la carpeta 'raw' si no existe
os.makedirs(CARPETA_RAW, exist_ok=True)

# Función para descomprimir archivos .zip
def descomprimir_archivos(carpeta_origen, carpeta_destino):
    archivos = [f for f in os.listdir(carpeta_origen) if f.endswith('.zip')]
    
    if not archivos:
        warnings.warn("No se encontraron archivos .zip para descomprimir.")
        return
    
    for archivo in archivos:
        ruta_archivo = os.path.join(carpeta_origen, archivo)
        print(f"Descomprimiendo {archivo} en {carpeta_destino}...")
        
        # Descomprimir archivo
        with zipfile.ZipFile(ruta_archivo, 'r') as zip_ref:
            zip_ref.extractall(carpeta_destino)
        print(f"{archivo} descomprimido exitosamente.")

# Verificar si la carpeta compressed existe
if os.path.exists(CARPETA_COMPRESSED):
    descomprimir_archivos(CARPETA_COMPRESSED, CARPETA_RAW)
else:
    warnings.warn(f"La carpeta {CARPETA_COMPRESSED} no fue encontrada. Verifique la estructura")

Descomprimiendo palmeras.zip en ./resources/datasets/raw...
palmeras.zip descomprimido exitosamente.


Una vez descomprimido los archivos, podemos observar que el conjunto está en formato PascalVOC (aunque también tiene dos archivos .csv que pueden utilizarse para otros formatos, ej: YOLO).

Podemos observar las imágenes y su detección:

## Machine learning model engineering

Como experimentos, dado que inicialmente el modelo no tiene como objetivo el funcionamiento en tiempo real, se plantea utilizar modelos de detección en dos etapas (ej: Faster R-CNN). Sin embargo, también se tienen en cuenta modelos de una sola etapa (ej: YOLO)

### Faster R-CNN

### YOLO

<!-- ## Quality assurance for machine learning applications -->

<!-- ## Deployment -->

<!-- ## Monitoring and maintenance -->

## Referencias

- Adel Ammar, and Anis Koubaa. (2023). Aerial images of palm trees [Data set]. Kaggle. https://doi.org/10.34740/KAGGLE/DSV/6382990